# Automatic Grammatical Error Correction using BERT and GPT2
In this notebook, we'll look at word probabilities, sentence probabilities and how both can assist with improved spell checking and automatic grammatical error correction.
This notebook fleshes out, expands and clarifies some ideas mentioned in:
* `The Unreasonable Effectiveness of Transformer Language Models in Grammatical Error Correction` by Dimitris Alikaniotis, Vipul Raheja [https://arxiv.org/abs/1906.01733]
* `BERT has a Mouth, and It Must Speak: BERT as a Markov Random Field Language Model` by Alex Wang, Kyunghyun Cho [https://arxiv.org/pdf/1902.04094.pdf]

### TL,DR;
* "We extract the probability of a sentence from BERT, by iteratively masking every word in the sentence and then summing the log probabilities. While this approach is far from ideal, it has been shown (Wang and Cho, 2019) that it approximates the log-likelihood of a sentence."

### We'll be using free versions of LanguageTool to spot problem areas:
Open a terminal, install docker, and then run:

`docker pull erikvl87/languagetool`

`docker run --rm -p 8010:8010 erikvl87/languagetool`

This will allow you to hit the grammar check endpoint in sections of the code below

#### The paid version is undoubtedly worth it.

We'll call the service and check for problems to fix using the following function:

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
LOG=logging.getLogger("probas")
logging.basicConfig(level=logging.ERROR)
logging.disable(logging.INFO)
import json
from pprint import pprint
import requests
from itertools import chain

from transformers import BertTokenizer, BertForMaskedLM, GPT2Tokenizer, GPT2LMHeadModel

# Add parent dir, so we can access our common code
import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from mlyoucanuse.bert_fun import (
    get_alternate_words, 
    get_word_probabilities,
    get_word_in_sentence_probability, 
    sum_log_probabilities)
from mlyoucanuse.gpt2_fun import predict_next_token

In [3]:
def check_sentence(sentence):
    """Helper method to check sentences using languagetool"""
    res = requests.post('http://localhost:8010/v2/check', data= f"language=en-US&text={sentence}")    
    obj = json.loads(res.content.decode('utf-8'))
    return obj.get('matches')

In [4]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-large-cased')
bert_model = BertForMaskedLM.from_pretrained("bert-large-cased-whole-word-masking")
bert_model.eval()
logging.disable(logging.NOTSET)
LOG.info('Done!')

I0220 00:28:30.160527 4581219776 <ipython-input-4-89a9450b03ab>:5] Done!


# The follow example sentence has an error, can you spot it?

In [5]:
error_sent_1 = 'I am looking forway to see you.' 

In [6]:
pprint(check_sentence(error_sent_1))

[{'context': {'length': 6,
              'offset': 13,
              'text': 'I am looking forway to see you.'},
  'contextForSureMatch': 0,
  'ignoreForIncompleteSentence': False,
  'length': 6,
  'message': 'Possible spelling mistake found',
  'offset': 13,
  'replacements': [{'value': 'Norway'},
                   {'value': 'foray'},
                   {'value': 'for way'}],
  'rule': {'category': {'id': 'TYPOS', 'name': 'Possible Typo'},
           'description': 'Possible spelling mistake',
           'id': 'MORFOLOGIK_RULE_EN_US',
           'issueType': 'misspelling'},
  'sentence': 'I am looking forway to see you.',
  'shortMessage': 'Spelling mistake',
  'type': {'typeName': 'Other'}}]


# The 4th word is a problem, let's mask it and see BERT's top 5 predictions

In [7]:
get_alternate_words(error_sent_1, word_number=4, bert_tokenizer=bert_tokenizer, bert_model=bert_model, top=5)

[('up', 0.5737241506576538),
 ('forward', 0.27456167340278625),
 ('down', 0.038530636578798294),
 ('back', 0.01723042130470276),
 ('happy', 0.015939364209771156)]

# Our winner is in second place, perhaps BERT can look at the whole context and give a better decision

In [8]:
tmp = error_sent_1.split()
tmp[3] = 'up'
res = get_word_probabilities(' '.join(tmp), bert_tokenizer=bert_tokenizer, bert_model=bert_model)
tmp, res, sum_log_probabilities(res)

(['I', 'am', 'looking', 'up', 'to', 'see', 'you.'],
 (('I', ('I',), (0.9985413551330566,)),
  ('am', ('am',), (0.08395612984895706,)),
  ('looking', ('looking',), (0.006572773680090904,)),
  ('up', ('up',), (0.5737241506576538,)),
  ('to', ('to',), (0.9950821399688721,)),
  ('see', ('see',), (0.26060330867767334,)),
  ('you', ('you',), (0.00461436016485095,)),
  ('.', ('.',), (0.9470862150192261,))),
 21.999381840501748)

In [9]:
tmp = error_sent_1.split()
tmp[3] = 'forward'
res= get_word_probabilities(' '.join(tmp), bert_tokenizer=bert_tokenizer, bert_model=bert_model)
tmp, res, sum_log_probabilities(res)

(['I', 'am', 'looking', 'forward', 'to', 'see', 'you.'],
 (('I', ('I',), (0.9981794357299805,)),
  ('am', ('am',), (0.4198238253593445,)),
  ('looking', ('looking',), (0.9980196952819824,)),
  ('forward', ('forward',), (0.27456167340278625,)),
  ('to', ('to',), (0.9999183416366577,)),
  ('see', ('see',), (0.0017120845150202513,)),
  ('you', ('you',), (0.07673513889312744,)),
  ('.', ('.',), (0.9251171350479126,))),
 25.66170175304526)

# BOOM, context is king

### Let's check Language Tool's suggestions against BERT

In [10]:
bert_tokenizer.tokenize('foray')

['for', '##ay']

In [11]:
bert_tokenizer.encode('foray', add_special_tokens=False)

[1111, 4164]

In [12]:
tmp = error_sent_1.split()
tmp[3] = 'foray'
res = get_word_probabilities(' '.join(tmp), bert_tokenizer=bert_tokenizer, bert_model=bert_model)
tmp, res, sum_log_probabilities(res)

(['I', 'am', 'looking', 'foray', 'to', 'see', 'you.'],
 (('I', ('I',), (0.997231662273407,)),
  ('am', ('am',), (0.16062967479228973,)),
  ('looking', ('looking',), (0.0022117383778095245,)),
  ('foray', ('for', '##ay'), (0.07856044173240662, 1.936624443032997e-07)),
  ('to', ('to',), (0.9439206719398499,)),
  ('see', ('see',), (0.033311937004327774,)),
  ('you', ('you',), (0.33044958114624023,)),
  ('.', ('.',), (0.9541561603546143,))),
 10.886311394188859)

In [13]:
tmp = error_sent_1.split()
tmp[3] = 'for way'
res = get_word_probabilities(' '.join(tmp), bert_tokenizer=bert_tokenizer, bert_model=bert_model)
tmp, res, sum_log_probabilities(res)

(['I', 'am', 'looking', 'for way', 'to', 'see', 'you.'],
 (('I', ('I',), (0.9958599209785461,)),
  ('am', ('am',), (0.1605089008808136,)),
  ('looking', ('looking',), (0.7438622713088989,)),
  ('for', ('for',), (3.939509952033404e-06,)),
  ('way', ('way',), (0.003746665082871914,)),
  ('to', ('to',), (0.9703219532966614,)),
  ('see', ('see',), (0.04287201166152954,)),
  ('you', ('you',), (0.16430877149105072,)),
  ('.', ('.',), (0.9581384062767029,))),
 16.257300041524807)

In [14]:
tmp = error_sent_1.split()
tmp[3] = 'Norway'
res = get_word_probabilities(' '.join(tmp), bert_tokenizer=bert_tokenizer, bert_model=bert_model)
tmp, res, sum_log_probabilities(res)

(['I', 'am', 'looking', 'Norway', 'to', 'see', 'you.'],
 (('I', ('I',), (0.9986119270324707,)),
  ('am', ('am',), (0.13397079706192017,)),
  ('looking', ('looking',), (0.00010536557238083333,)),
  ('Norway', ('Norway',), (1.1956956313952105e-08,)),
  ('to', ('to',), (0.9202854633331299,)),
  ('see', ('see',), (0.01011697482317686,)),
  ('you', ('you',), (0.00849774107336998,)),
  ('.', ('.',), (0.9501535296440125,))),
 -2.0658867724002343)

In [15]:
from mlyoucanuse.gpt2_fun import predict_next_token

# GPT2 takes a shot

In [16]:
logging.disable(logging.INFO)
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')
gpt2_model.eval()
logging.disable(logging.NOTSET)

In [17]:
predict_next_token('I am looking ', gpt2_model, gpt2_tokenizer)

(('forward', 0.3665640652179718),
 ('for', 0.35346919298171997),
 ('to', 0.08423731476068497))

# GPT2 may be more accurate because it was trained a lot more data, however:
* it can only predict next token
* it doesn't provide a masked word prediction like BERT

# Some classes of errors we'll cover below:
* Real word spelling errors; e.g. not OOV
* Errors involving a missing word
* Errors of an extra word
* Errors of agreement
* Errors of verb form

# Real word spelling errors; e.g. not OOV

In [18]:
err_7 = "We can order then directly from the web."
err_7c = "We can order them directly from the web."

get_alternate_words(err_7, word_number=4, bert_tokenizer=bert_tokenizer, bert_model=bert_model, top=5)

[('them', 0.2891930639743805),
 ('it', 0.2877180278301239),
 ('these', 0.034848809242248535),
 ('everything', 0.03310466185212135),
 ('this', 0.028820807114243507)]

In [19]:
err_7_res = get_word_probabilities(err_7, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
err_7, err_7_res, sum_log_probabilities(err_7_res) 

('We can order then directly from the web.',
 (('We', ('We',), (0.1670190542936325,)),
  ('can', ('can',), (0.15270596742630005,)),
  ('order', ('order',), (0.0013151239836588502,)),
  ('then', ('then',), (3.236498741898686e-05,)),
  ('directly', ('directly',), (0.0045602815225720406,)),
  ('from', ('from',), (0.4603978991508484,)),
  ('the', ('the',), (0.9359720349311829,)),
  ('web', ('web',), (9.6403155112057e-06,)),
  ('.', ('.',), (0.970289945602417,))),
 2.9934637969274336)

In [20]:
err_7c_res = get_word_probabilities(err_7c, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
err_7c, err_7c_res, sum_log_probabilities(err_7c_res)

('We can order them directly from the web.',
 (('We', ('We',), (0.11701209098100662,)),
  ('can', ('can',), (0.6486707925796509,)),
  ('order', ('order',), (0.0020630417857319117,)),
  ('them', ('them',), (0.2891930639743805,)),
  ('directly', ('directly',), (0.5851895213127136,)),
  ('from', ('from',), (0.5881755948066711,)),
  ('the', ('the',), (0.8397545218467712,)),
  ('web', ('web',), (0.00017660936282481998,)),
  ('.', ('.',), (0.9594199657440186,))),
 21.51979759853427)

# Failure: sadly BERT doesn't know yoga

In [21]:
# real word spelling error; e.g. not OOV
err_6 = "Yoga brings peace and vitality to you life."
err_6c = "Yoga brings peace and vitality to your life."

err_6_res = get_word_probabilities(err_6, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
err_6c_res = get_word_probabilities(err_6c, bert_tokenizer=bert_tokenizer, bert_model=bert_model)

get_alternate_words(err_6, word_number=7, bert_tokenizer=bert_tokenizer, bert_model=bert_model, top=5)

[('human', 0.4340229332447052),
 ('daily', 0.27425795793533325),
 ('all', 0.13561923801898956),
 ('everyday', 0.040244799107313156),
 ('a', 0.036808937788009644)]

In [22]:
get_word_in_sentence_probability(err_6, 'your', bert_model, bert_tokenizer,  position=7)

(9.532024058955813e-09,)

In [23]:
get_word_in_sentence_probability(err_6, 'you', bert_model=bert_model, bert_tokenizer=bert_tokenizer, position=7)

(1.50075166516217e-07,)

In [24]:
err_6, err_6_res, sum_log_probabilities(err_6_res)

('Yoga brings peace and vitality to you life.',
 (('Yoga', ('Yoga',), (3.380464477231726e-05,)),
  ('brings', ('brings',), (0.7734541893005371,)),
  ('peace', ('peace',), (0.03682466223835945,)),
  ('and', ('and',), (0.9945341348648071,)),
  ('vitality',
   ('vital', '##ity'),
   (0.006828881334513426, 0.040755514055490494)),
  ('to', ('to',), (0.5773847103118896,)),
  ('you', ('you',), (1.3096871498419205e-06,)),
  ('life', ('life',), (9.335688810097054e-05,)),
  ('.', ('.',), (0.9591231346130371,))),
 0.5902883172001783)

In [25]:
err_6c, err_6c_res, sum_log_probabilities(err_6c_res)

('Yoga brings peace and vitality to your life.',
 (('Yoga', ('Yoga',), (9.048711945069954e-05,)),
  ('brings', ('brings',), (0.9053715467453003,)),
  ('peace', ('peace',), (0.11162364482879639,)),
  ('and', ('and',), (0.9985194802284241,)),
  ('vitality',
   ('vital', '##ity'),
   (0.027336645871400833, 0.10959531366825104)),
  ('to', ('to',), (0.7094688415527344,)),
  ('your', ('your',), (0.004815567284822464,)),
  ('life', ('life',), (0.21725884079933167,)),
  ('.', ('.',), (0.9960422515869141,))),
 21.42762560058551)

# Errors involving a missing word
(challenging to assess, but technically possible)

In [26]:
err_2 = "I'm not sure what I'm up tomorrow."
err_2c = "I'm not sure what I'm up to tomorrow."

err_2_res =  get_word_probabilities(err_2, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
err_2, err_2_res, sum_log_probabilities(err_2_res)

("I'm not sure what I'm up tomorrow.",
 (('I', ('I',), (0.9999358654022217,)),
  ("'m", ("'", 'm'), (0.9951947331428528, 0.9977612495422363)),
  ('not', ('not',), (0.9959115982055664,)),
  ('sure', ('sure',), (0.9941648840904236,)),
  ('what', ('what',), (0.29322049021720886,)),
  ('I', ('I',), (0.9999940395355225,)),
  ("'m", ("'", 'm'), (0.04088006541132927, 0.00016173844051081687)),
  ('up', ('up',), (2.425032107566949e-05,)),
  ('tomorrow', ('tomorrow',), (2.398638798695174e-08,)),
  ('.', ('.',), (0.980730414390564,))),
 13.899174085168003)

In [27]:
err_2c_res= get_word_probabilities( err_2c , bert_tokenizer=bert_tokenizer, bert_model=bert_model)
err_2c, err_2c_res, sum_log_probabilities(err_2c_res)

("I'm not sure what I'm up to tomorrow.",
 (('I', ('I',), (0.9999393224716187,)),
  ("'m", ("'", 'm'), (0.9969546794891357, 0.9985707998275757)),
  ('not', ('not',), (0.9961541295051575,)),
  ('sure', ('sure',), (0.994225800037384,)),
  ('what', ('what',), (0.9878880381584167,)),
  ('I', ('I',), (0.9999959468841553,)),
  ("'m", ("'", 'm'), (0.8839161396026611, 0.9024779200553894)),
  ('up', ('up',), (0.5902553796768188,)),
  ('to', ('to',), (0.5820885300636292,)),
  ('tomorrow', ('tomorrow',), (0.0015056264819577336,)),
  ('.', ('.',), (0.9864518642425537,))),
 52.034313559054446)

In [28]:
err_3 = "I am psychologist."
err_3c = "I am a psychologist."

err_3_res= get_word_probabilities(err_3, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
err_3, err_3_res, sum_log_probabilities(err_3_res)

('I am psychologist.',
 (('I', ('I',), (0.9858765006065369,)),
  ('am', ('am',), (0.6945590376853943,)),
  ('psychologist', ('psychologist',), (4.13914813179872e-06,)),
  ('.', ('.',), (0.8961634635925293,))),
 5.53732544369199)

In [29]:
err_3c_res= get_word_probabilities( err_3c , bert_tokenizer=bert_tokenizer, bert_model=bert_model)
err_3c, err_3c_res, sum_log_probabilities(err_3c_res)

('I am a psychologist.',
 (('I', ('I',), (0.9993333220481873,)),
  ('am', ('am',), (0.6896260380744934,)),
  ('a', ('a',), (0.987400233745575,)),
  ('psychologist', ('psychologist',), (0.0009351802873425186,)),
  ('.', ('.',), (0.9536884427070618,))),
 15.618708943693967)

# Errors of an extra word

In [30]:
err_4 = "Why is do they appear in this particular section?"
err_4c = "Why do they appear in this particular section?"

err_4_res= get_word_probabilities(err_4, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
err_4, err_4_res, sum_log_probabilities(err_4_res)

('Why is do they appear in this particular section?',
 (('Why', ('Why',), (0.16198164224624634,)),
  ('is', ('is',), (6.46511180093512e-05,)),
  ('do', ('do',), (7.278690645762254e-07,)),
  ('they', ('they',), (0.19688940048217773,)),
  ('appear', ('appear',), (0.2648281753063202,)),
  ('in', ('in',), (0.9802661538124084,)),
  ('this', ('this',), (0.2701944410800934,)),
  ('particular', ('particular',), (0.07991039007902145,)),
  ('section', ('section',), (0.0006710817688144743,)),
  ('?', ('?',), (0.9710816144943237,))),
 6.306634272865201)

In [31]:
err_4c_res= get_word_probabilities(err_4c, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
err_4c, err_4c_res, sum_log_probabilities(err_4c_res)

('Why do they appear in this particular section?',
 (('Why', ('Why',), (0.7230836749076843,)),
  ('do', ('do',), (0.5018960237503052,)),
  ('they', ('they',), (0.3515675365924835,)),
  ('appear', ('appear',), (0.25058338046073914,)),
  ('in', ('in',), (0.9878813028335571,)),
  ('this', ('this',), (0.5135653018951416,)),
  ('particular', ('particular',), (0.09514071047306061,)),
  ('section', ('section',), (0.0003888505743816495,)),
  ('?', ('?',), (0.9980717897415161,))),
 27.118407410420453)

In [32]:
err_5 = "Is our youth really in in such a state of disrepair?"
err_5c = "Is our youth really in such a state of disrepair?"

err_5_res= get_word_probabilities(err_5, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
err_5, err_5_res, sum_log_probabilities(err_5_res)

('Is our youth really in in such a state of disrepair?',
 (('Is', ('Is',), (0.5408534407615662,)),
  ('our', ('our',), (0.10689887404441833,)),
  ('youth', ('youth',), (1.3555451005231589e-05,)),
  ('really', ('really',), (0.004895591177046299,)),
  ('in', ('in',), (8.652749966131523e-05,)),
  ('in', ('in',), (0.02773268148303032,)),
  ('such', ('such',), (0.8100823760032654,)),
  ('a', ('a',), (0.9965115189552307,)),
  ('state', ('state',), (0.9993855953216553,)),
  ('of', ('of',), (0.9995771050453186,)),
  ('disrepair',
   ('di', '##s', '##re', '##pair'),
   (0.13752403855323792,
    0.048513058573007584,
    0.007498561404645443,
    0.012231351807713509)),
  ('?', ('?',), (0.9984367489814758,))),
 22.235348126878584)

In [33]:
err_5c_res= get_word_probabilities(err_5c, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
err_5c, err_5c_res, sum_log_probabilities(err_5c_res)

('Is our youth really in such a state of disrepair?',
 (('Is', ('Is',), (0.9543763399124146,)),
  ('our', ('our',), (0.12623824179172516,)),
  ('youth', ('youth',), (7.61529736337252e-06,)),
  ('really', ('really',), (0.11097059398889542,)),
  ('in', ('in',), (0.9997593760490417,)),
  ('such', ('such',), (0.99972003698349,)),
  ('a', ('a',), (0.9996761083602905,)),
  ('state', ('state',), (0.9997681975364685,)),
  ('of', ('of',), (0.9999357461929321,)),
  ('disrepair',
   ('di', '##s', '##re', '##pair'),
   (0.07216285169124603,
    0.030781781300902367,
    0.008856026455760002,
    0.009444918483495712)),
  ('?', ('?',), (0.9991154074668884,))),
 32.87163616218337)

# Errors of Agreement

In [34]:
err_8 = "I awaits your response."
err_8c = "I await your response."

err_8_res= get_word_probabilities(err_8, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
err_8, err_8_res, sum_log_probabilities(err_8_res)

('I awaits your response.',
 (('I', ('I',), (0.0034313967917114496,)),
  ('awaits',
   ('a', '##wai', '##ts'),
   (0.0020877665374428034, 0.5089987516403198, 2.2291551431408152e-05)),
  ('your', ('your',), (0.005505810026079416,)),
  ('response', ('response',), (0.03314106538891792,)),
  ('.', ('.',), (0.9816052317619324,))),
 0.37563092539703113)

In [35]:
err_8c_res= get_word_probabilities(err_8c, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
err_8c, err_8c_res, sum_log_probabilities(err_8c_res)

('I await your response.',
 (('I', ('I',), (0.7317708730697632,)),
  ('await',
   ('a', '##wai', '##t'),
   (0.0020877665374428034, 0.5089987516403198, 0.012238427996635437)),
  ('your', ('your',), (0.014312163926661015,)),
  ('response', ('response',), (0.014434893615543842,)),
  ('.', ('.',), (0.9868567585945129,))),
 12.175776196388547)

In [36]:
err_9 = "The first of these scientist begin in January."
err_9c = "The first of these scientists begin in January."
err_9_res = get_word_probabilities(err_9, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
err_9, err_9_res, sum_log_probabilities(err_9_res)

('The first of these scientist begin in January.',
 (('The', ('The',), (0.9965081810951233,)),
  ('first', ('first',), (0.004373201169073582,)),
  ('of', ('of',), (0.9866845011711121,)),
  ('these', ('these',), (0.057628609240055084,)),
  ('scientist', ('scientist',), (1.0894053437127127e-09,)),
  ('begin', ('begin',), (0.010585297830402851,)),
  ('in', ('in',), (0.9678221344947815,)),
  ('January', ('January',), (0.0036275957245379686,)),
  ('.', ('.',), (0.9448446035385132,))),
 2.2490826196956517)

In [37]:
err_9c_res = get_word_probabilities(err_9c, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
err_9c, err_9c_res, sum_log_probabilities(err_9c_res)

('The first of these scientists begin in January.',
 (('The', ('The',), (0.9947749376296997,)),
  ('first', ('first',), (0.0005680953036062419,)),
  ('of', ('of',), (0.91595858335495,)),
  ('these', ('these',), (0.05670550465583801,)),
  ('scientists', ('scientists',), (8.002268145901326e-07,)),
  ('begin', ('begin',), (0.0017818311462178826,)),
  ('in', ('in',), (0.8879834413528442,)),
  ('January', ('January',), (0.001778475008904934,)),
  ('.', ('.',), (0.9370677471160889,))),
 4.126118188968048)

# Errors with Verb Form

In [38]:
err_10 = "Brent would often became stunned by resentment."
err_10c = "Brent would often become stunned by resentment."
err_10_res= get_word_probabilities(err_10, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
err_10, err_10_res, sum_log_probabilities(err_10_res)

('Brent would often became stunned by resentment.',
 (('Brent', ('Brent',), (5.147402043803595e-05,)),
  ('would', ('would',), (0.0011423516552895308,)),
  ('often', ('often',), (0.0006808876059949398,)),
  ('became', ('became',), (1.685629649728071e-05,)),
  ('stunned', ('stunned',), (0.00016873223648872226,)),
  ('by', ('by',), (0.32671085000038147,)),
  ('resentment', ('resentment',), (7.069127605063841e-05,)),
  ('.', ('.',), (0.9559879899024963,))),
 -17.498713297129644)

In [39]:
err_10c_res= get_word_probabilities(err_10c, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
err_10c, err_10c_res, sum_log_probabilities(err_10c_res)

('Brent would often become stunned by resentment.',
 (('Brent', ('Brent',), (6.225280958460644e-05,)),
  ('would', ('would',), (0.372565895318985,)),
  ('often', ('often',), (0.022015467286109924,)),
  ('become', ('become',), (0.028336457908153534,)),
  ('stunned', ('stunned',), (8.714829164091498e-05,)),
  ('by', ('by',), (0.3506300151348114,)),
  ('resentment', ('resentment',), (5.4146814363775775e-05,)),
  ('.', ('.',), (0.9466897249221802,))),
 -1.4844211944607055)

In [40]:
err_11 = "I having mostly been moving flat."
err_11c = "I have mostly been moving flat."
err_11_res= get_word_probabilities(err_11, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
err_11, err_11_res, sum_log_probabilities(err_11_res)

('I having mostly been moving flat.',
 (('I', ('I',), (0.005292738322168589,)),
  ('having', ('having',), (6.289655721047893e-05,)),
  ('mostly', ('mostly',), (0.005705267656594515,)),
  ('been', ('been',), (0.5970264077186584,)),
  ('moving', ('moving',), (0.00012258144852239639,)),
  ('flat', ('flat',), (9.995254913519602e-06,)),
  ('.', ('.',), (0.8984378576278687,))),
 -8.98863935388643)

In [41]:
err_11c_res= get_word_probabilities(err_11c, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
err_11c, err_11c_res, sum_log_probabilities(err_11c_res)

('I have mostly been moving flat.',
 (('I', ('I',), (0.11393590271472931,)),
  ('have', ('have',), (0.41972973942756653,)),
  ('mostly', ('mostly',), (0.005898055154830217,)),
  ('been', ('been',), (0.9094554781913757,)),
  ('moving', ('moving',), (0.00014484622806776315,)),
  ('flat', ('flat',), (9.372281965625007e-06,)),
  ('.', ('.',), (0.9684615731239319,))),
 3.5182476527900794)

# That's all for now!